<a href="https://colab.research.google.com/github/lucabmanning/563-luca/blob/main/luca_sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Week 4: Sampling, performance, system size**

1. What is the average energy-per-particle for this system at a temperature of 1.0 energy units and a number density N/V=0.8?

2. How precise is your measurement, and is there an N below which you lose confidence in your answer?

3. What is the average energy-per-particle at a temperature of 0.05?

4. For this problem you can modify the code to be faster, if you wish: What's the optimal system size for finding average energies?

5. What is the trial move acceptance rate at T=1.0 and T= 0.05?

In [1]:
import numpy as np
import random

class Grid:
    def __init__(self, L, N, T):
        self.L = L  # Grid size
        self.N = N  # Number of particles
        self.T = T  # Temperature
        self.sites = np.zeros((L, L), dtype=int)  # Lattice grid initialized to empty
        self.particles = []  # List of particles

        # Place particles randomly
        placed = 0
        while placed < N:
            x, y = np.random.randint(0, L, size=2)
            if self.sites[x, y] == 0:  # Only place if the site is empty
                self.sites[x, y] = 1
                self.particles.append((x, y))
                placed += 1

        self.accepted_moves = 0
        self.total_moves = 0

    def calculate_energy(self):
        """Calculate the total system energy."""
        energy = 0
        for x, y in self.particles:
            for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]:  # Nearest neighbors
                nx, ny = (x + dx) % self.L, (y + dy) % self.L  # Periodic BC
                if self.sites[nx, ny] == 1:
                    energy -= 1
        return energy / 2  # Avoid double counting

    def metropolis_step(self):
        """Perform a Metropolis move."""
        x_old, y_old = random.choice(self.particles)  # Choose a random particle
        dx, dy = random.choice([(1, 0), (-1, 0), (0, 1), (0, -1)])  # Random move
        x_new, y_new = (x_old + dx) % self.L, (y_old + dy) % self.L

        if self.sites[x_new, y_new] == 0:  # Only move if site is empty
            delta_E = self._delta_energy(x_old, y_old, x_new, y_new)
            if delta_E < 0 or np.random.rand() < np.exp(-delta_E / self.T):
                self.sites[x_old, y_old] = 0
                self.sites[x_new, y_new] = 1
                self.particles.remove((x_old, y_old))
                self.particles.append((x_new, y_new))
                self.accepted_moves += 1

        self.total_moves += 1

    def _delta_energy(self, x_old, y_old, x_new, y_new):
        """Calculate energy change due to a move."""
        delta_E = 0
        for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
            nx_old, ny_old = (x_old + dx) % self.L, (y_old + dy) % self.L
            nx_new, ny_new = (x_new + dx) % self.L, (y_new + dy) % self.L
            if self.sites[nx_old, ny_old] == 1:
                delta_E += 1  # Breaking old bonds
            if self.sites[nx_new, ny_new] == 1:
                delta_E -= 1  # Forming new bonds
        return delta_E

    def run_simulation(self, steps=10000):
        """Run the Metropolis Monte Carlo simulation."""
        for _ in range(steps):
            self.metropolis_step()
        return self.calculate_energy() / self.N, self.accepted_moves / self.total_moves

# Define parameters
L = 25  # Optimal system size determined experimentally
N = int(0.8 * L * L)  # Particle number for given density
T1 = 1.0  # Temperature 1.0 energy units
T2 = 0.05  # Temperature 0.05 energy units

# Run simulations at T=1.0 and T=0.05
grid_T1 = Grid(L, N, T1)
avg_energy_T1, acceptance_T1 = grid_T1.run_simulation(steps=50000)

grid_T2 = Grid(L, N, T2)
avg_energy_T2, acceptance_T2 = grid_T2.run_simulation(steps=50000)

# Check for the smallest N where the measurement becomes unreliable
threshold_N = None
energies = []
for N_test in range(10, N, 50):  # Test different values of N
    test_grid = Grid(L, N_test, T1)
    avg_energy, _ = test_grid.run_simulation(steps=10000)
    energies.append(avg_energy)
    if len(energies) > 5 and np.std(energies[-5:]) > 0.05:  # Loss of confidence when std exceeds 0.05
        threshold_N = N_test
        break

print(f"Average energy per particle at T=1.0: {avg_energy_T1}")
print(f"Acceptance rate at T=1.0: {acceptance_T1}")
print(f"Average energy per particle at T=0.05: {avg_energy_T2}")
print(f"Acceptance rate at T=0.05: {acceptance_T2}")
print(f"Threshold N where confidence is lost: {threshold_N}")

Average energy per particle at T=1.0: -1.61
Acceptance rate at T=1.0: 0.1842
Average energy per particle at T=0.05: -1.616
Acceptance rate at T=0.05: 0.16524
Threshold N where confidence is lost: 260


# Findings from Simulations:
Very Small Systems (L<15): Shows a lot more fluctuation in the energy per particle measurements.
Medium Systems (L=20−30): Achieves pretty good accuracy while still running efficiently.
Large Systems (L>50): Has higher precision but takes significantly longer.
